In [1]:
#V2: Remove earlystopping, remove parameters: monitor_loss, loss_mode, patience
from cnn_method import *
from cnn_preproc_function import *
from inference_function import *

import optparse
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing

import keras
import tensorflow as tf

from keras_model_configuration import *
from keras_metric import *

import datetime
import time
import os
import shutil



Using TensorFlow backend.


In [2]:
# path = "/media/workstation/Storage/DL_Output/C001_D0H0M30_S_4bug/ASSETS_MODELS/EWZ_1541096508"
# path = "/media/workstation/Storage/DL_Output/C001_D0H0M30_S_4bug/ASSETS_MODELS/IAU_1541096508"
# path = "/media/workstation/Storage/DL_Output/C001_D0H0M30_S_4bug/ASSETS_MODELS/VWO_1541096508"
path = "/media/workstation/Storage/DL_Output/C001_D0H0M30_S_4bug/ASSETS_MODELS/XLI_1541096508"




# path = "/media/workstation/Storage/DL_Output/C005_D0H0M15_S/ASSETS_MODELS/EURJPY_1541384216"

In [3]:
DATA_PARAMS = joblib.load(os.path.join(path, "DATA_PARAMS.pkl"))
MODEL_PARAMS = joblib.load(os.path.join(path, "MODEL_PARAMS.pkl"))

In [4]:
DATA_PARAMS

array(['EURUSD', 'EEM', 'EFA', 'EWZ', 'FXI', 'HYG', 'IAU', 'IWM', 'SPY',
       'VWO', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XOP', 'JNK',
       'IYR', 'VEA', 'SLV', 'XLV', 'RSX', 'TLT', 'EWJ', 'OIH', 'GLD',
       'EZU', 'KRE', 'SMH', 'XLB', 'XRT', 'LQD', 'EWT', 'XLY', 'VNQ',
       'EWH', 'EWW', 'XBI', 'DIA', 'EWG', 'VGK', 'IEF', 'EMB', 'FEZ',
       'AGG', 'EWC', 'USDJPY', 'AUDUSD', 'USDCAD', 'EURJPY', 'NZDUSD'],
      dtype=object)

In [5]:
MODEL_PARAMS

{'device': '0',
 'keras_model_function': 'rnn_model_conf_1_best',
 'BATCH_SIZE': 256,
 'EPOCHS': 900,
 'LEARNING_RATE': 5e-05,
 'training_number': 'C001',
 'training_code': 'C001_D0H0M30_S',
 'root_output_folder': '/media/workstation/9EB4ABE9B4ABC1DF/DL_Output',
 'INIT_TIME': '1541096508',
 'ASSET_FOLDER_NAME': 'XLI_1541096508',
 'outputmain_folder': '/media/workstation/9EB4ABE9B4ABC1DF/DL_Output/C001_D0H0M30_S/ASSETS_MODELS',
 'collect_signal_folder': '/media/workstation/9EB4ABE9B4ABC1DF/DL_Output/C001_D0H0M30_S/SIGNAL',
 'collect_bestmodel_folder': '/media/workstation/9EB4ABE9B4ABC1DF/DL_Output/C001_D0H0M30_S/BESTMODEL',
 'project_folder': '/media/workstation/9EB4ABE9B4ABC1DF/DL_Output/C001_D0H0M30_S/ASSETS_MODELS/XLI_1541096508',
 'models_folder': '/media/workstation/9EB4ABE9B4ABC1DF/DL_Output/C001_D0H0M30_S/ASSETS_MODELS/XLI_1541096508/models',
 'logs_folder': '/media/workstation/9EB4ABE9B4ABC1DF/DL_Output/C001_D0H0M30_S/ASSETS_MODELS/XLI_1541096508/logs'}

In [6]:
gpudevice = str(MODEL_PARAMS["device"])
os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"]=gpudevice
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [7]:
raw_data_file = "./DATA/PRICE_LIQUIDASSET_30_MIN.csv"
TARGET_TO_PREDICT = "XLI"
BREAKOUT_WINDOW = 100
FUTURE_PERIOD_PREDICT = 5
TARGET_FUNCTION = "cumulative_returns"
TARGET_THRESHOLD = -0.001
FLIP = True
end_split = [datetime.datetime(2011,1,1), datetime.datetime(2013,1,1), datetime.datetime(2017,1,1)]
SEQ_LEN = 20
BATCH_SIZE = 128
collect_signal_folder = "./output"
models_folder = os.path.join(path, "models")

In [8]:
print("====== PREPROCESSING =======")
t0 = time.time()
df = load_data(raw_data_file)
df = clean_data_breakout_x(df, target_col = TARGET_TO_PREDICT, breakout_window = BREAKOUT_WINDOW)

====== PREPROCESSING =======
Load Data: Done!


In [9]:
df = create_target_2(df, "target", FUTURE_PERIOD_PREDICT, TARGET_FUNCTION)
df = classify_target(df, "target", TARGET_THRESHOLD, FLIP)
df, X, Y, start_index, end_index, scaler = split_df(df, end_split, scale = True)
train_data_gen, val_data_gen, val_2_data_gen, test_data_gen, shape_x = TSGenerator(X, Y, SEQ_LEN, BATCH_SIZE, start_index, end_index)
class_weights = get_class_weights(df, start_index, end_index)
    
    
print("====== INFERENCE =======")
# init_dir(collect_signal_folder)
# init_dir(collect_bestmodel_folder)
#INFERENCE, PRED SIGNAL
df, timestamp, all_data_gen = FullTSGeneratorDirect(df, X, Y, SEQ_LEN, batch_size = 64)

(128, 20, 52)
Number of batches per epoch: 69
TSGenerator: Done!
[0.84402528 1.22669079]
====== INFERENCE =======


In [10]:

#Pick best model
mdf = parse_model_folder(models_folder)
fmdf, best_model_file = best_model_score(mdf)
best_model_path = os.path.join(models_folder, best_model_file)
model = keras.models.load_model(best_model_path, custom_objects=None, compile=False)
print("Best Model: Done! --", best_model_file)

Best Model: Done! -- DL-0123-1.4324-0.5121-0.3736-0.3872.model


In [11]:
INIT_TIME = 0
#Prediction
t0 = time.time()
y = model.predict_generator(all_data_gen)
t1 = time.time()
print("Prediction: Done!", t1-t0, "seconds!")

#Signal Output
signal_df = pd.DataFrame(dict(Date = timestamp, signal_raw = y.flatten())).set_index("Date")
signal_file = "signal_" + TARGET_TO_PREDICT + "_" + str(FLIP) + "_" + str(INIT_TIME) + ".csv"
signal_df.to_csv(os.path.join(collect_signal_folder, signal_file))
print("Prop:", np.mean(signal_df["signal_raw"] > 0.5))
print("Signal Output: Done!")

Prediction: Done! 2.5967493057250977 seconds!
Prop: 0.42977878615995463
Signal Output: Done!


In [12]:










# shutil.copy2(best_model_path, project_folder)
# print("Copy best model: Done!")

# shutil.copy2(best_model_path, os.path.join(collect_bestmodel_folder, TARGET_TO_PREDICT + "_" + best_model_file))
# print("Collect best model: Done!")

# signal_df.to_csv(os.path.join(collect_signal_folder, signal_file))
# print("Collect signal.csv: Done!")